# Install dependencies

In [5]:


%pip install ollama
%pip install openai
%pip install transformers
%pip install tiktoken
%pip install huggingface-hub
%pip install python-dotenv



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Prepare data

In [1]:
import sys
sys.path.append('../')  # Add the path to the my_packages module
from my_packages.data_processing.split_dataset import split_on_shots, read_dataset_to_json
from my_packages.data_processing.get_labels_data import used_libraries_from_dataset, used_libraries_to_string
from my_packages.analysis.analyze_datasets import analyze_library_distribution, analyze_instance_distribution, analyze_visual_node_types_distribution

main_dataset_folder = '../data/mbpp_transformed_code_examples/sanitized-MBPP-midio.json'
DATA_DIR = '../data/mbpp_transformed_code_examples/only_files'

dataset = read_dataset_to_json(main_dataset_folder)
    
num_shot = 10 # Few-shot examples
eval_size_percentage = 0.5

train_data, val_data, test_data = split_on_shots(num_shot, eval_size_percentage, dataset, seed = 64, write_to_file=True)

# Extract all unique nodes (library_functions) across datasets
used_libraries_json = used_libraries_from_dataset(train_data)
explained_used_libraries = used_libraries_to_string(used_libraries_json)

#Bar chart of distribuation
# analyze_library_distribution(train_data, val_data, test_data)
# analyze_instance_distribution(train_data, val_data, test_data)
# analyze_visual_node_types_distribution(train_data, val_data, test_data)

15
7
8
Library functions included in the dataset: 32


# Prepare prompts

In [2]:
from my_packages.prompting.few_shot import split_and_format, create_few_shot_messages

train_prompts, train_responses = split_and_format(train_data,"CODE_TEMPLATE",'../data/mbpp_transformed_code_examples/only_files/')
val_prompts, val_responses = split_and_format(val_data,"CODE_TEMPLATE", '../data/mbpp_transformed_code_examples/only_files/')
test_prompts, test_responses = split_and_format(test_data ,"CODE_TEMPLATE", '../data/mbpp_transformed_code_examples/only_files/')

few_shot_messages = create_few_shot_messages(explained_used_libraries, train_prompts, train_responses, "CODE_GENERATOR_TEMPLATE")


# Open source models

Use OpenAI python library, but connect it to ollama to use open source models

In [3]:
import ollama
from openai import OpenAI
host = 'http://localhost:11434'

open_models = [
    #llama models:
    # {"name": "llama3.1:8b-instruct-fp16"},
    # {"name": "llama3.3:70b-instruct-q8_0"},
    # {"name": "llama3.3:70b-instruct-fp16"},
    {"name": "codestral:22b-v0.1-q8_0"},


    # {"name": "qwq:32b-preview-fp16"},
    # {"name": "mistral-large:123b-instruct-2407-q4_K_M"},
]

### Calculate max tokens for each model

In [4]:
from my_packages.utils.tokens_utils import find_max_tokens_code_tokenizer, find_max_tokens_code_api

client = OpenAI(
  base_url=host+'/v1/',
  api_key='ollama' #Ignored with 'ollama'. To use GPT models, this key need to be set.
)

# To be able to pull (download) models from ollama in the code, ollama.pull('model_name'). Not possible with the openAI client.
ollama.Client(
  host=host,
)

## TODO: Add the code as static json so i can use it in the future and dont need to run this code again
open_models_to_test = []
for model_info in open_models:
    # ollama.pull(model_info["name"])
    print(f"Model: {model_info['name']}")
    max_tokens = find_max_tokens_code_api(DATA_DIR, model_info["name"], client.embeddings)
    model_result = {
        "name": model_info["name"],
        "max_tokens": max_tokens,
    }
    open_models_to_test.append(model_result)

print(open_models_to_test)

Model: codestral:22b-v0.1-q8_0
File: task_id_8.midio | Tokens: 543
File: task_id_14.midio | Tokens: 1435
File: task_id_23.midio | Tokens: 1318
File: task_id_34.midio | Tokens: 725
File: task_id_16.midio | Tokens: 1717
File: task_id_25.midio | Tokens: 806
File: task_id_11.midio | Tokens: 791
File: task_id_18.midio | Tokens: 108
File: task_id_32.midio | Tokens: 885
File: task_id_28.midio | Tokens: 93
File: task_id_17.midio | Tokens: 1452
File: task_id_21.midio | Tokens: 585
File: task_id_27.midio | Tokens: 1089
File: task_id_33.midio | Tokens: 808
File: task_id_37.midio | Tokens: 602
File: task_id_41.midio | Tokens: 744
File: task_id_6.midio | Tokens: 593
File: task_id_38.midio | Tokens: 881
File: task_id_2.midio | Tokens: 105
File: task_id_10.midio | Tokens: 370
File: task_id_3.midio | Tokens: 512
File: task_id_31.midio | Tokens: 465
File: task_id_40.midio | Tokens: 984
File: task_id_19.midio | Tokens: 523
File: task_id_7.midio | Tokens: 534
File: task_id_24.midio | Tokens: 567
File: ta

# GPT-models

In [5]:
gpt_models = [
    # {"name": "o1-preview", "tokenization": "o200k_base"},
    {"name": "gpt-4o", "tokenization": "o200k_base"},
]

### Calculate max tokens for each model

In [6]:
import tiktoken

gpt_models_to_test = []
for model_info in gpt_models:
    encoding = tiktoken.encoding_for_model(model_info["name"])
    max_tokens = find_max_tokens_code_tokenizer(DATA_DIR, encoding)
    model_result = {
        "name": model_info["name"],
        "max_tokens": max_tokens
    }
    gpt_models_to_test.append(model_result)
print(gpt_models_to_test)

File: task_id_8.midio | Tokens: 353
File: task_id_14.midio | Tokens: 900
File: task_id_23.midio | Tokens: 839
File: task_id_34.midio | Tokens: 478
File: task_id_16.midio | Tokens: 1064
File: task_id_25.midio | Tokens: 516
File: task_id_11.midio | Tokens: 509
File: task_id_18.midio | Tokens: 71
File: task_id_32.midio | Tokens: 633
File: task_id_28.midio | Tokens: 67
File: task_id_17.midio | Tokens: 892
File: task_id_21.midio | Tokens: 388
File: task_id_27.midio | Tokens: 784
File: task_id_33.midio | Tokens: 558
File: task_id_37.midio | Tokens: 440
File: task_id_41.midio | Tokens: 564
File: task_id_6.midio | Tokens: 382
File: task_id_38.midio | Tokens: 672
File: task_id_2.midio | Tokens: 64
File: task_id_10.midio | Tokens: 256
File: task_id_3.midio | Tokens: 332
File: task_id_31.midio | Tokens: 341
File: task_id_40.midio | Tokens: 741
File: task_id_19.midio | Tokens: 338
File: task_id_7.midio | Tokens: 344
File: task_id_24.midio | Tokens: 382
File: task_id_15.midio | Tokens: 739
File: ta

# Evaluation

### Open source models

In [8]:
from my_packages.evaluation.code_accuracy import run_code_evaluation, evaluate_code_accuracy
from my_packages.utils.server_utils import server_diagnostics, is_remote_server_reachable
from openai import OpenAI

host = 'http://localhost:11434'
if is_remote_server_reachable(url = host + "/api/tags"):
    print("Server is reachable.")
else:
    server_diagnostics()
    print("Ollama server is not reachable. Batch job might have finished. Try running bash script again.")

client = OpenAI(
  base_url=host+'/v1/',
  api_key='ollama' #Ignored with 'ollama'. To use GPT models, this key need to be set.
)


results = {}
for model in open_models_to_test:
    result = run_code_evaluation(
        client,
        few_shot_messages,
        model,
        val_prompts,
        val_responses,
        test_prompts,
        test_responses,
        temperatures=[0.2, 0.5, 1.0],
        top_ps=[0.2, 0.5, 0.8],
        seeds=[3, 75, 346]
    )
    results[model["name"]] = result
      
# Print the final results
print("\nFinal Results:")
for model_name, runs in results.items():
    for run in runs:
        print(f"Model: {model_name}")
        seed = run["seed"]
        val_syn = run["validation_syntax_accuracy"]
        val_sem = run["validation_semantic_accuracy"]
        test_syn = run["test_syntax_accuracy"]
        test_sem = run["test_semantic_accuracy"]
        best_temperature = run["temperature"]
        best_top_p = run["top_p"]
        print(
            f"  Seed {seed}: "
            f"Val Syntax Acc: {val_syn:.2f}, "
            f"Val Semantic Acc: {val_sem:.2f}, "
            f"Test Syntax Acc: {test_syn:.2f}, "
            f"Test Semantic Acc: {test_sem:.2f}, "
            f"Best temperature: {best_temperature:.2f},"
            f"Best top_p: {best_top_p:.2f},"
        )

# print(few_shot_messages)



Server is reachable.
VALIDATION Phase
Validating with temperature: 0.2, and top_p: 0.2


KeyboardInterrupt: 

### GPT models

In [3]:

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv("../.env")
openai_token = os.getenv('OPENAI_API_KEY')

if openai_token:
    client = OpenAI(api_key=openai_token)
    
    results = {}
    for model in gpt_models_to_test:
        print(f"Model: {model['name']}")
        result = run_code_evaluation(
            client,
            few_shot_messages,
            model,
            val_prompts,
            val_responses,
            test_prompts,
            test_responses,
            temperatures=[0.2, 0.5, 0.8],
            top_ps=[0.2, 0.5, 1.0],
            seeds=[3, 75, 346]
        )
        results[model["name"]] = result
        
    # Print the final results
    print("\nFinal Results:")
    for model_name, runs in results.items():
        for run in runs:
            print(f"Model: {model_name}")
            seed = run["seed"]
            val_syn = run["validation_syntax_accuracy"]
            val_sem = run["validation_semantic_accuracy"]
            test_syn = run["test_syntax_accuracy"]
            test_sem = run["test_semantic_accuracy"]
            best_temperature = run["temperature"]
            best_top_p = run["top_p"]
            print(
                f"  Seed {seed}: "
                f"Val Syntax Acc: {val_syn:.2f}, "
                f"Val Semantic Acc: {val_sem:.2f}, "
                f"Test Syntax Acc: {test_syn:.2f}, "
                f"Test Semantic Acc: {test_sem:.2f}, "
                f"Best temperature: {best_temperature:.2f},"
                f"Best top_p: {best_top_p:.2f},"
            )
else:
    print("OPENAI_API_KEY is not set in your environment variables.")

NameError: name 'gpt_models_to_test' is not defined